# LDA 모델

### 1. 데이터 불러오기

In [1]:
import pandas as pd

# 시 데이터 불러오기
poem = pd.read_csv("../data/nlp_poem.csv", index_col=False)
poem.head()

,author,title,content
0,이순,가을에는 이런 사람이 그립다,수평선이나 지평선 보다<br>더 깊고 아득한 눈빛으로<br>세상의 그리움 다 깨워서...
1,문정희,가을 노트,<br><br><br>그대 떠나간 후<br>나의 가을은<br>조금만 건드려도<br>...
2,이순,가을비 오는 날은,<br>자정이 되어 간신히 그치는 비<br>간신히 버리는 그리움<br>그동안 너무 ...
3,이성선,가을 편지,<br>잎이 떨어지고 있습니다<br>원고지처럼 하늘이 한 칸씩<br>비어 가고 있습...
4,박제영,가을에는,가을에는 잠시 여행을 떠날 일이다<br>그리 수선스러운 준비는 하지 말고<br>그리...


## 2. 데이터 전처리

In [2]:
# 불용어
stopwords = [
    '하다', '이다', '아니다', '않다', '되다', '있다', '없다', '싶다', '알다', '보다', '마다', '같다', '지다', '가다',
    '나', '그대', '당신', '우리', '이', '그', '저', '무엇', '이런', '저런', '그런', '것',
    '하나', '모든', '때문', '다음', '위해', '사람', '줄', '못', '그리다', '하소', '가장', '많다'
]

In [3]:
from konlpy.tag import Okt

# 토큰화 함수
def tokenize(text):
    okt = Okt()

    # 줄바꿈(<br>) 제거
    text = text.replace("<br>", " ")

    # 형태소(품사) 분석 - 정규화有, 어간추출有
    okt_pos = okt.pos(text, norm=True, stem=True)

    # 특정 품사(명사, 동사, 형용사)만 사용
    lst_word = [word for word, pos in okt_pos if pos in ["Noun", "Adjective", "Verb"]]

    # 불용어 제거
    lst_word = [word for word in lst_word if word not in stopwords]

    return lst_word

In [4]:
# 시 내용에 토큰화 적용
content = poem["content"]
content_tokens = content.apply(tokenize)
content_tokens.head()

0    [수평선, 지평선, 더, 깊다, 아득하다, 눈빛, 세상, 그리움, 깨우다, 다른, ...
1    [떠나가다, 후, 가을, 조금, 건드리다, 우수수, 몸, 떨다, 다한, 말, 다한,...
2    [자정, 되어다, 그치다, 비, 버리다, 그리움, 그동안, 많다, 버리다, 허다, ...
3    [잎, 떨어지다, 원고지, 하늘, 칸, 비어, 빈, 곳, 맑은, 영혼, 잉크, 물,...
4    [가을, 잠시, 여행, 떠나다, 일이, 수선스럽다, 준비, 가깝다, 멀다, 데, 가...
Name: content, dtype: object

In [5]:
from gensim import corpora

dictionary = corpora.Dictionary(content_tokens)
corpus = [dictionary.doc2bow(text) for text in content_tokens]

C:\Programming\Miniconda\miniconda3\envs\nlp3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [10]:
import gensim

NUM_TOPICS = 20  # 20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(
    corpus, num_topics=NUM_TOPICS, id2word=dictionary, passes=15
)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.017*"매혹" + 0.008*"어머니" + 0.007*"비둘기" + 0.007*"상자"')
(1, '0.021*"내" + 0.017*"돌다" + 0.015*"자다" + 0.014*"어깨"')
(2, '0.017*"내" + 0.014*"묻다" + 0.012*"길" + 0.012*"나무"')
(3, '0.048*"말" + 0.019*"새" + 0.017*"심장" + 0.016*"뛰다"')
(4, '0.035*"발길질" + 0.035*"약초" + 0.019*"농" + 0.019*"끼울다"')
(5, '0.080*"길" + 0.065*"글자" + 0.045*"밟다" + 0.035*"바꾸다"')
(6, '0.031*"불씨" + 0.019*"달려오다" + 0.018*"꿈" + 0.017*"아침"')
(7, '0.012*"새" + 0.011*"더" + 0.010*"되어다" + 0.009*"입"')
(8, '0.027*"때" + 0.027*"쥐" + 0.020*"햇볕" + 0.017*"숨"')
(9, '0.042*"사과" + 0.021*"산" + 0.021*"쓴다" + 0.020*"동쪽"')
(10, '0.016*"사랑" + 0.015*"내" + 0.011*"수" + 0.010*"말"')
(11, '0.027*"소리" + 0.017*"오다" + 0.017*"빠르다" + 0.015*"소녀"')
(12, '0.030*"꽃" + 0.020*"눈" + 0.013*"를" + 0.012*"오전"')
(13, '0.026*"의자" + 0.025*"오르막" + 0.010*"얼음" + 0.003*"만세"')
(14, '0.047*"길이" + 0.034*"시" + 0.030*"나다" + 0.026*"곳"')
(15, '0.047*"내" + 0.027*"쓰다" + 0.024*"거" + 0.017*"묻다"')
(16, '0.016*"시간" + 0.013*"생각" + 0.013*"계단" + 0.011*"내"')
(17, '0.020*"속" + 0.018*"살다" + 0.017*"너"

In [12]:
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

C:\Programming\Miniconda\miniconda3\envs\nlp3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
